# BlockSci Setup

In [ ]:
import blocksci
import matplotlib.pyplot as plt
import matplotlib.ticker
import collections
import pandas as pd
import numpy as np
%matplotlib notebook

In [ ]:
# parser_data_directory should be set to the data-directory which the blocksci_parser output
chain = blocksci.Blockchain(parser_data_directory)

# Measuring different types of address use

In [ ]:
%time net_coins_per_block = chain.map_blocks(lambda block: block.net_address_type_value())

In [ ]:
df = pd.DataFrame(net_coins_per_block).fillna(0).cumsum()/1e8
df = chain.heights_to_dates(df)
df = df.rename(columns={t:blocksci.address_type.pretty_name(t) for t in df.columns})
ax = df.resample("W").mean().plot()
ax.set_ylim(ymin=0)

# Visualizing Transaction Fees for an individual block

In [ ]:
example_block_height = 465100
%time df = pd.DataFrame(chain[example_block_height].fees_per_byte, columns=["Satoshis per byte"])
ax = df.reset_index().plot.scatter(x="index", y="Satoshis per byte")
ax.set_ylim(0)
ax.set_xlim(0)
plt.show()

# Identifying Overlay Applications

In [ ]:
%time txes = chain.script_type_txes(0, len(chain), blocksci.nulldata)

In [ ]:
%time labels = [(tx.block.time, blocksci.label_application(tx)) for tx in txes]

In [ ]:
df = pd.DataFrame(labels, columns=["date", "label"])
df = df.reset_index().groupby(["date", "label"]).count().unstack(level=-1).fillna(0)
df.columns = df.columns.droplevel()
important_columns = list(df[df.index > pd.to_datetime("1-1-2016")].sum().sort_values()[-10:].index)
important_columns = [x for x in important_columns if "Address" not in x]
ax = df[df.index > pd.to_datetime("1-1-2016")].cumsum().resample("w").mean()[important_columns].plot()
ax.set_ylim(0)
plt.tight_layout()

# Average Fees per Block in July in USD

In [ ]:
%time fees = [(block.time, sum(block.fees)) for block in chain.range('Jul 2017')]

In [ ]:
converter = blocksci.CurrencyConverter()

In [ ]:
df = pd.DataFrame(fees, columns=["Date", "Fee"])
df.index = df["Date"]
del df["Date"]
df = converter.satoshi_to_currency_df(df, chain)
ax = df.resample("d").mean().plot(legend=False)
ax.set_ylim(ymin=0)
plt.tight_layout()

# Transactions Paying over 1000 dollars in fees

In [ ]:
%time high_fee_txes = chain.cpp.filter_tx("fee(tx) > 10000000", 0, len(chain))

In [ ]:
df = pd.DataFrame([(tx.block.height, tx.fee) for tx in high_fee_txes], columns=["height", "fee"])
df.index = df["height"]
df["fee"] = df.apply(lambda x: converter.satoshi_to_currency(x["fee"], chain[x["height"]].time), axis=1)
df = df[df["fee"] > 1000]
df = chain.heights_to_dates(df)
fig, ax = plt.subplots()
ax.plot_date(df.index, df["fee"], fmt="x")
ax.set_yscale("log")
formatter = matplotlib.ticker.ScalarFormatter(useOffset=False)
formatter.set_scientific(False)
ax.yaxis.set_major_formatter(formatter)
plt.tight_layout()